# Preprocessing

In this notebook, we will be preprocessing the data for machine learning models.

***

# Import libraries + Scripts

In [3]:
# Setting PYTHONHASHSEED
# Run this block, then restart the kernel
import os

os.environ['PYTHONHASHSEED'] = '0'

In [1]:
# TESTING PYTHONHASHSEED
assert(hash('hi') == hash('hi'))

In [2]:
# Importing libraries
import sys
import os

# To import script
sys.path.append('py_scripts')
import A_eda as A

***